In [ ]:
# Create a sample list

In [ ]:
import time

In [ ]:
sample_list = [ 'the','and','have','that','for','you','with','say','this'
,'they','but','his','from','not']

import random
import time
import pandas as pd

value = random.randint(5,10)
csv_list = []

for i in range(value):
  csv_list.append(sample_list[i])

file_name = "words" + str(time.ctime()) + ".csv"
df = pd.DataFrame(csv_list)
df.to_csv(file_name,index=False)

In [ ]:
# Spark Streaming
# Data received from a server listening on TCP socket.

In [ ]:
! pip install pyspark

In [ ]:
# import necessary modules

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.types import StructType

spark = SparkSession \
    .builder \
    .appName("WordCount") \
    .getOrCreate()

In [ ]:
# Create Dataframe - Stream of input toward a host

userSchema = StructType().add("value", "string")

lines = spark \
        .readStream \
        .format("csv") \
        .schema(userSchema) \
        .load(r'/content/wordsWed Feb  7 04:40:21 2024.csv')


In [ ]:
# split the lines into words
words = lines.select(explode(split(lines.value,"\n")).alias("word"))

In [ ]:
wordCounts = words.groupBy("word").count()
words.writeStream \
.format("csv") \
.option("checkpointLocation", "path/to/checkpoint/dir") \
.option("path", "path/to/destination/dir") \
.start()

In [ ]:
outfilename = str(time.ctime) + "out.csv"

wordCounts.writeStream \
    .queryName("aggregates") \
    .outputMode("complete") \
    .format("memory") \
    .start()

In [ ]:
spark.sql("select * from aggregates").show()